# Autor: Frogmaker

**Inicjaca bibliotek i pliku źródłowego**

In [1]:
from openpyxl import load_workbook
from openpyxl import Workbook

In [2]:
wb = load_workbook('exercises/data/f1-results.xlsx')

**Zadanie 1:** Przygotuj arkusz o nazwie *Standings*, który będzie zawierał sumaryczną liczbę punktów zawodników po każdym wyścigu. 
    
Arkusz powinien zawierać tabelą jak poniżej:

|                  | Australia | China | ... |
|------------------|-----------|-------|-----|
| Lewis Hamilton   | 18        | 43    | ... |
| Sebastian Vettel | 25        | 43    | ... |
| ...              |           |       |     |

**Zadanie 2:** Na podstawie powyższej tabeli, w arkuszu *Standings* umieść wykres liniowy (`LineChart`), który zaprezentuje sumaryczną liczbę punktów po kolejnych wyścigach. Na osi X powinny być nazwy wyścigów, na osi Y liczba punktów, a serie reprezentują poszczególnych kierowców. 

## Zadanie 1
**Kod pobierający nazwy arkuszy**

In [3]:
sheetnames = wb.sheetnames

**Kod punktacji i sortowania zawodników**

In [4]:
rank = {}
for ws in wb:
    for row in ws.iter_rows():
        driver = row[2].value
        current_points = int(row[5].value)
        if driver in rank:
            rank[driver] += current_points
        else:
            rank[driver] = current_points
            

sorted_rank = sorted(rank.items(), key=lambda x:x[1], reverse = True)            

**Utworzenie nowego arkusza w nowym pliku:**

In [5]:
new_wb = Workbook()
new_ws = new_wb.active
new_ws.title = "Standings"

**Utworzenie ramki danych (lista kierowców, lista wyścigów)**

In [6]:
new_wb.active = new_wb['Standings']

new_ws = new_wb.active

#wypełnienie nagłówków kolumn miejscami wyścigów

for i in range(len(sheetnames)):
    new_ws.cell(row=1, column=i+2).value = sheetnames[i]

#wypełnienie pierwszej kolumny nazwiskami kierowców w odpowiedniej kolejności

for i in range(len(sorted_rank)):
    new_ws.cell(row = 2+i, column=1).value = sorted_rank[i][0][:-3]

**Uzupełnienie tabeli danymi**

In [7]:
#wypełnienie poszczególnych komórek wynikami kierowców pojedyńczych wyścigów

for col in  range(len(sheetnames)):
    for row in range(len(sorted_rank)):
        new_ws.cell(row  = row + 2, column = col + 2).value = 0 #pozbysie się komórek NaN
        ws.active = wb[new_ws.cell(row = 1, column = col+2).value]
        for cell in ws.active.iter_rows():            
            driver = cell[2].value[:-3]
            if driver == new_ws.cell(row = row+2,column = 1).value:        
                new_ws.cell(row  = row + 2, column = col + 2).value = int(cell[5].value)            
                
#sumowanie kolumn

for row in range(len(sorted_rank)):
    for col in  range(len(sheetnames)-1):        
        new_ws.cell(row = row+2, column = col +3).value += int(new_ws.cell(row = row+2, column = col +2).value)

**Zapisanie nowego pliku z danymi**

In [8]:
new_wb.save('./exercises/data/standings.xlsx')

**Otwarcie pliku w celu sprawdzenia otrzymanego wyniku**

In [ ]:
import os
os.system("start EXCEL.EXE  exercises/data/standings.xlsx")

## Zadanie 2

**Import bibliotek potrzebnych do narysowania wykresu**

In [9]:
from openpyxl import Workbook, load_workbook
from openpyxl.chart import (
    LineChart,
    Reference,
    Series
)
from openpyxl.chart.axis import DateAxis

**Zdefiniowanie słownika przypisującego kierowcom numer wiersza**

In [10]:
drivers = {}
i = 2
for item in sorted_rank:
    drivers[item[0][:-3]] = i
    i = i+1

**Rysowanie wykresu**

In [11]:
wb = load_workbook("./exercises/data/standings.xlsx")
ws = wb["Standings"]

LC = LineChart()
LC.title = "F1_seasone2014"
#LC.style = 13
LC.y_axis.title = 'Points'
LC.x_axis.title = 'Race'

for item in drivers:    
    data = Reference(ws, min_col=2, min_row=drivers[item], max_col=21, max_row=drivers[item])
    serie = Series(data, title=item)
    LC.append(serie)

data = Reference(ws, min_col=2, min_row=1, max_col=21, max_row=1)
LC.set_categories(data)

ws.add_chart(LC, "W2")


In [12]:
# zapis pliku
wb.save("./exercises/data/standings.xlsx")

In [13]:
# otwarcie pliku
import os
os.system("start EXCEL.EXE  exercises/data/standings.xlsx")

0

# Koniec zadania